In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\Medical-Image-Analysis\\notebooks'

In [3]:
import os 

os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\Medical-Image-Analysis'

In [39]:
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [6]:
from dotenv import load_dotenv
load_dotenv() 

True

In [7]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/fraidoon_omarzai/Medical-Image-Analysis.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "fraidoon_omarzai"
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv('DAGSHUB_TOKEN')

In [8]:
mlflow.set_tracking_uri("https://dagshub.com/fraidoon_omarzai/Medical-Image-Analysis.mlflow")

In [9]:
# Specify the name of the experiment
experiment_name = "medimgExperiments"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

In [10]:
experiment_id

'3'

In [22]:
# Query the best run based on a specific metric (e.g., 'accuracy')
best_run = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.val_accuracy DESC"],  # Change 'accuracy' to your metric
    max_results=1
)

best_run_id = best_run.iloc[0].run_id

In [23]:
best_run

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_accuracy,metrics.final_train_accuracy,metrics.val_loss,metrics.final_train_loss,params.batch size,params.epochs,tags.mlflow.log-model.history,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.type
0,ca65ea02b90b46c09e1789ee1c2d8cca,3,FINISHED,mlflow-artifacts:/85bcc57ae96845f3a8e18e9b7965...,2024-07-14 10:18:58.723000+00:00,2024-07-14 10:24:20.631000+00:00,0.8,0.795,0.478436,0.424722,32,20,"[{""run_id"": ""ca65ea02b90b46c09e1789ee1c2d8cca""...",383f200ab2878b689adfc135e41f26024fe800f2,fraidoon_omarzai,Transfer_learning_Model,.\src\pipeline\stage_03_training_evaluation.py,LOCAL


In [24]:
best3_run_id

'ca65ea02b90b46c09e1789ee1c2d8cca'

In [25]:
model_uri = f"runs:/{best_run_id}/mlflow_model.keras"
model = mlflow.tensorflow.load_model(model_uri)

In [37]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling          │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,572,584 (17.44 MB)

 Trainable params: 174,337 (681.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 348,676 (1.33 MB)

In [38]:
model.save('mlflow_model.keras')

**Model staging**

In [27]:
from src.constants import *

In [29]:
model_name = REGISTERED_MODEL_NAME

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == best_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        run_id = mv["run_id"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Production"
        )
    else:
        current_version = mv["version"]
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Staging"
        )  

{   'aliases': [],
    'creation_timestamp': 1720952658608,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1720952658608,
    'name': 'EfficientNetB0',
    'run_id': 'ca65ea02b90b46c09e1789ee1c2d8cca',
    'run_link': '',
    'source': 'mlflow-artifacts:/85bcc57ae96845f3a8e18e9b7965bcf3/ca65ea02b90b46c09e1789ee1c2d8cca/artifacts/mlflow_model.keras',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}
